# Calculating Similarity

create some transformer embedded vectors, then use a cosine similarity

In [1]:
import h
import pandas as pd
pd.set_option('display.max_colwidth', None)
# use movies dataset
df = pd.read_csv('../data/imdb_top_1000.csv')#.head(10)
df['title_overview'] = df['Series_Title'] + ' ' + df['Overview']
df[['Series_Title', 'Overview']].head(10)

,Series_Title,Overview
0,The Shawshank Redemption,"Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency."
1,The Godfather,An organized crime dynasty's aging patriarch transfers control of his clandestine empire to his reluctant son.
2,The Dark Knight,"When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice."
3,The Godfather: Part II,"The early life and career of Vito Corleone in 1920s New York City is portrayed, while his son, Michael, expands and tightens his grip on the family crime syndicate."
4,12 Angry Men,A jury holdout attempts to prevent a miscarriage of justice by forcing his colleagues to reconsider the evidence.
5,The Lord of the Rings: The Return of the King,Gandalf and Aragorn lead the World of Men against Sauron's army to draw his gaze from Frodo and Sam as they approach Mount Doom with the One Ring.
6,Pulp Fiction,"The lives of two mob hitmen, a boxer, a gangster and his wife, and a pair of diner bandits intertwine in four tales of violence and redemption."
7,Schindler's List,"In German-occupied Poland during World War II, industrialist Oskar Schindler gradually becomes concerned for his Jewish workforce after witnessing their persecution by the Nazis."
8,Inception,A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into the mind of a C.E.O.
9,Fight Club,"An insomniac office worker and a devil-may-care soapmaker form an underground fight club that evolves into something much, much more."


In [2]:
# sentences = df['Overview']
# h.compute_similarity(0, sentences)

# df['similar_movies'] = pd.Series(df.index).apply(lambda x: h.compute_similarity(x, df['Overview']))
# df['similar_movies']

In [3]:
sentence_embeddings = h.sentence_transformer_similarity(df['title_overview'])
sentence_embeddings

array([[-0.42897072,  0.39082983,  1.4352702 , ...,  0.8727772 ,
        -0.27758625, -0.52901405],
       [-0.31057072,  0.7643628 ,  0.72591037, ..., -0.5141307 ,
        -0.23639445, -0.29770607],
       [-0.5239642 ,  0.2013199 ,  0.32214215, ..., -0.29536867,
        -0.31208324, -0.51214534],
       ...,
       [-0.6130735 ,  0.62725097, -0.10875668, ..., -0.10114583,
         0.1711134 , -0.06715756],
       [-0.5834191 ,  0.05922156, -0.9729282 , ..., -0.59030604,
         0.17073274, -0.07679442],
       [-0.3590858 ,  0.66717935,  0.75014377, ..., -1.3990557 ,
        -0.4917294 ,  0.39107534]], dtype=float32)

In [4]:
from sklearn.metrics.pairwise import cosine_similarity
def list_similarity_scores(main_index, embeddings):
    similarity_scores = cosine_similarity(
        [sentence_embeddings[main_index]],
        sentence_embeddings)
    ranked_list = sorted([(i, x) for i, x in enumerate(similarity_scores[0])], key=lambda x: x[1], reverse=True)
    return ranked_list[:10]

In [16]:
df['similarity_scores'] = pd.Series(df.index).apply(lambda x: list_similarity_scores(x, df['Overview']))
df['similarity_scores'][0]

[(0, 0.99999976),
 (532, 0.70702296),
 (763, 0.70188177),
 (391, 0.68108857),
 (267, 0.677529),
 (282, 0.6744362),
 (167, 0.6739683),
 (203, 0.67307293),
 (88, 0.65801036),
 (505, 0.65431154)]

In [15]:
df['similar_movies'] = pd.Series(df.index).apply(lambda y: [(df['Series_Title'][int(x)], df['Overview'][int(x)]) for x in [x[0] for x in df['similarity_scores'][y]]])
df['similar_movies'][0]

[('The Shawshank Redemption',
  'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.'),
 ('The Blues Brothers',
  'Jake Blues, just released from prison, puts together his old band to save the Catholic home where he and his brother Elwood were raised.'),
 ('This Is England',
  'A young boy becomes friends with a gang of skinheads. Friends soon become like family, and relationships will be pushed to the very limit.'),
 ('Sling Blade',
  'Karl Childers, a simple man hospitalized since his childhood murder of his mother and her lover, is released to start a new life in a small town.'),
 ('Stand by Me',
  'After the death of one of his friends, a writer recounts a childhood journey with his friends to find the body of a missing boy.'),
 ('Zerkalo',
  'A dying man in his forties remembers his past. His childhood, his mother, the war, personal moments and things that tell of the recent history of all the Russian nation.'),
 (

In [8]:
df[['similar_movies', 'similarity_scores', 'Series_Title',
 'Released_Year',
 'Certificate',
 'Runtime',
 'Genre',
 'IMDB_Rating',
 'Overview',
 'Meta_score',
 'Director',
 'Star1',
 'Star2',
 'Star3',
 'Star4',
 'No_of_Votes',
 'Gross',
 'Poster_Link']].to_csv('similar_movies_df.csv')